In [15]:
import numpy as np
import re
import string
import pandas as pd
import nltk
from nltk.collocations import *
from pattern.en import lemma
import requests
import json
import time
import hashlib
import uuid
import csv
import phrasemachine
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.request import Request
import difflib
from nltk.stem import WordNetLemmatizer 
from wechatpy.client import WeChatClient
import random
import http.client
import stardict
from stardict import LemmaDB
lemma = LemmaDB()
lemma.load('lemma.en.txt')
import urllib

In [16]:
with open("master_word_list.json",'r') as load_f:
    master_word_dict = json.load(load_f)

In [17]:
# Easy word list
CET4_list = []

with open('CET-4_Vocab_v2.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        line=line.lower()
        if line.startswith(('a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z')):
            word = re.findall('([a-z]+)',line)
            CET4_list.append(word[0].lower().rstrip())
        else:
            continue

highschool_list = []

with open('Highschool_Vocab.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        highschool_list.append(line.lower().rstrip())  

middleschool_list = []

with open('MiddleSchool_Vocab.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        middleschool_list.append(line.lower().rstrip())        
        
        
elementary_list = []

with open('Elementary_Vocab.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        elementary_list.append(line.lower().rstrip())

easy_word_set = set(CET4_list) | set(highschool_list) | set(elementary_list) | set(middleschool_list)
easy_word_list = list(easy_word_set)

In [18]:
lemma_easy_word_list = []

for i in range(0,len(easy_word_list)):
    lemma_word = lemma.get(easy_word_list[i], reverse = False)
    if lemma_word == None:
        lemma_easy_word_list.append(easy_word_list[i])
    else:
        for j in range(0,len(lemma_word)):
            lemma_easy_word_list.append(lemma_word[j])
        lemma_easy_word_list.append(easy_word_list[i])
            



In [19]:
# word category
GMAT_list = []

with open('GMAT_edited.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        GMAT_list.append(line.lower().rstrip())
    
stardict = pd.read_csv("stardict.csv",encoding = 'utf-8')


# Output v0

In [20]:
bookname = input('enter the name of the book')

enter the name of the bookSomething I've Been Meaning to - Alice Munro


In [21]:
with open(bookname + ".txt", 'r',encoding='unicode_escape') as file:
    text = file.read().rstrip()

# remove puntuation
word_list_1 = re.findall("[a-zA-Z]+\'?\-?\'?[a-zA-Z]+", text)


#remove names
name_list = []
for i in word_list_1:
    if i.startswith(('A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z')) and i.lower() not in word_list_1:
        name_list.append(i)


word_list = []
for i in word_list_1:
    if i not in name_list:
        word_list.append(i.lower())
            
#Lemmatization
#LemmaDB can't lemma didn't or wouldn't, etc.
lemma_word_list = []

for i in range(0,len(word_list)):
    lemma_word = lemma.get(word_list[i], reverse = True)
    if lemma_word == None:
        lemma_word_list.append(word_list[i])
    else:
        lemma_word_list.append(lemma_word[0])


#select unfamiliar words
raw_word_list = []
for word in lemma_word_list:
    if word not in lemma_easy_word_list:
        raw_word_list.append(word)
    else:
        continue    


#count the word
word_count = pd.value_counts(raw_word_list)
data = pd.DataFrame({'Vocabulary':word_count.index, 'counts':word_count.values})
        
# point out where the replicate comes from
def get_key (dict, value):
    return [k for k, v in dict.items() if v == value]


for i in range(0,len(word_count.index)):
    for j in list(master_word_dict.values()):
        if data.loc[i,'Vocabulary'] in j:
            data.loc[i,'Replicate'] = get_key(master_word_dict,j)
        else:
            pass
for i in range(0,len(word_count.index)):
    try:
        category = re.findall("[a-z]+\-?[a-z]+", stardict.loc[stardict['word']== data.loc[i,'Vocabulary'] ,'tag'].to_string())
        if data.loc[i,'Vocabulary'] in GMAT_list:
            category.append('gmat')
        cate = ",".join(category)
        data.loc[i,'Cate'] = cate
    except IndexError:
        pass     
data.to_csv( bookname + "vocabularycounts_v0.csv",index=False,header=True)            